In [5]:
import pandas as pd

# Path to the CSV file with embeddings
embeddings_csv_path = 'outputs/FASTA_embeddings_20240213.csv'

# Load the CSV file. Assuming no header, specify column names manually
# The first column is 'TC_Number', followed by embedding dimensions
col_names = ['TC_Number'] + [f'dim{i}' for i in range(1, 1281)]
embeddings_df = pd.read_csv(embeddings_csv_path, header=None, names=col_names)

# Convert the dataframe to a dictionary {TC_Number: embedding_vector}
embeddings_dict = embeddings_df.set_index('TC_Number').T.to_dict('list')

In [ ]:
import glob
import os

# Original directory containing your positive/negative CSV files
original_files_dir = 'Positive_Negative/'

# New directory to save updated CSV files
updated_files_dir = 'Positive_Negative_Mapping/'

# Ensure the updated_files_dir exists, create if it doesn't
if not os.path.exists(updated_files_dir):
    os.makedirs(updated_files_dir)

# Iterate over each CSV file in the original directory
for csv_file in glob.glob(f'{original_files_dir}*.csv'):
    # Load the positive/negative CSV file
    pos_neg_df = pd.read_csv(csv_file, header=None, names=['positive', 'negative'])

    # Map the 'positive' and 'negative' TC_Numbers to their embeddings
    # Here, just storing a placeholder for the embedding reference
    pos_neg_df['positive_embedding'] = pos_neg_df['positive'].map(
        lambda x: 'Embedding_Ref' if x in embeddings_dict else 'Not Available')
    pos_neg_df['negative_embedding'] = pos_neg_df['negative'].map(
        lambda x: 'Embedding_Ref' if x in embeddings_dict else 'Not Available')

    # Construct the new file path in the updated directory
    # os.path.basename(csv_file) gets the file name from the original path
    new_file_path = os.path.join(updated_files_dir, os.path.basename(csv_file))

    # Save the updated DataFrame to the new file path
    pos_neg_df.to_csv(new_file_path, index=False, header=False)

    print(f'Updated file saved: {new_file_path}')

In [5]:
def convert_to_fasta(input_file, output_file):
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            parts = line.strip().split()  # Split the line into parts based on whitespace
            if len(parts) >= 2:  # Make sure there are at least two columns
                identifier = parts[0]  # First column is the identifier
                sequence = parts[1]  # Second column is the sequence
                # Write to output file in FASTA format
                outfile.write(f'>{identifier}\n{sequence}\n')

# Example usage
input_file = '/home/miservilla/esm/gene_sequence_homo_sapiens_no_dups.txt'  # The path to your input file
output_file = 'gene_sequence_homo_sapiens.fasta'  # The path to your output FASTA file
convert_to_fasta(input_file, output_file)


In [ ]:
def find_duplicates_txt(input_file):
    seen = {}
    duplicates = {}
    with open(input_file, 'r') as infile:
        for line in infile:
            identifier = line.strip().split()[0]  # Assuming the identifier is the first item
            if identifier in seen:
                if identifier in duplicates:
                    duplicates[identifier] += 1
                else:
                    duplicates[identifier] = 2  # Initialize with 2 since it's the first duplicate
            else:
                seen[identifier] = 1
    if duplicates:
        print("Duplicate identifiers found:")
        for dup, count in duplicates.items():
            print(f"{dup}: {count} times")
        print(f"Total number of duplicate entries: {sum(duplicates.values())}")
    else:
        print("No duplicates found.")

# Example usage
input_file = '/home/miservilla/esm/gene_sequence_homo_sapiens.txt'  # The path to your input text file
find_duplicates_txt(input_file)


In [4]:
def remove_duplicates_txt(input_file, output_file):
    seen = set()
    with open(input_file, 'r') as infile, open(output_file, 'w') as outfile:
        for line in infile:
            identifier = line.strip().split()[0]
            if identifier not in seen:
                outfile.write(line)
                seen.add(identifier)

# Example usage
input_file = '/home/miservilla/esm/gene_sequence_homo_sapiens.txt'  # Path to your input text file
output_file = 'gene_sequence_homo_sapiens_no_dups.txt'  # Path to your output text file without duplicates
remove_duplicates_txt(input_file, output_file)
